In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# from plotly.subplots import make_subplots # Não utilizado neste script final

# Configurações iniciais
plt.style.use('ggplot')
sns.set_palette("pastel")

# Carregar os dados (certifique-se que 'dados_limpos.csv' está no mesmo diretório ou forneça o caminho completo)
try:
    df = pd.read_csv('dados_limpos.csv')
except FileNotFoundError:
    print("Erro: O arquivo 'dados_limpos.csv' não foi encontrado. Verifique o caminho do arquivo.")
    exit()

# Pré-processamento dos salários
def convert_salary(salary_range):
    if isinstance(salary_range, str): # Adicionado para tratar possíveis NaNs que podem ser float
        if 'Acima' in salary_range:
            return 40001 # Considerando como mínimo do último intervalo
        ranges = {
            'Menos de R$ 1.000/mês': 500,
            'de R$ 1.001/mês a R$ 2.000/mês': 1500,
            'de R$ 2.001/mês a R$ 3.000/mês': 2500,
            'de R$ 3.001/mês a R$ 4.000/mês': 3500,
            'de R$ 4.001/mês a R$ 6.000/mês': 5000,
            'de R$ 6.001/mês a R$ 8.000/mês': 7000,
            'de R$ 8.001/mês a R$ 12.000/mês': 10000,
            'de R$ 12.001/mês a R$ 16.000/mês': 14000,
            'de R$ 16.001/mês a R$ 20.000/mês': 18000,
            'de R$ 20.001/mês a R$ 25.000/mês': 22500,
            'Acima de R$ 40.001/mês': 40001 # Chave específica para o valor mais alto
        }
        return ranges.get(salary_range, np.nan)
    return np.nan # Retorna NaN se não for string (ex: já é NaN)

df['Faixa_salarial_num'] = df['Faixa salarial mensal'].apply(convert_salary)

# Preparar DataFrame numérico e limpar valores infinitos para o heatmap
numeric_cols_for_heatmap = ['Faixa_salarial_num', 'Oportunidade de aprendizado', 'Reputação da empresa']
numeric_df_heatmap = df[numeric_cols_for_heatmap].copy()
numeric_df_heatmap.replace([np.inf, -np.inf], np.nan, inplace=True)

# 1. Distribuição Salarial
plt.figure(figsize=(12,6))
sns.histplot(df['Faixa_salarial_num'].dropna(), bins=20, kde=True)
plt.title('Distribuição de Salários Mensais')
plt.xlabel('Salário Médio (R$)')
plt.ylabel('Contagem')
plt.savefig('distribuicao_salarial.png')
plt.show()
plt.close()

# 2. Salário vs Experiência
plt.figure(figsize=(12,6))
sns.boxplot(x='Tempo de experiência na área de dados', y='Faixa_salarial_num', data=df)
plt.title('Relação entre Salário e Tempo de Experiência')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('salario_vs_experiencia.png')
plt.show()
plt.close()

# 3. Salário vs Formação Acadêmica
plt.figure(figsize=(14,7))
sns.violinplot(x='Nível de ensino alcançado', y='Faixa_salarial_num', data=df)
plt.title('Distribuição Salarial por Nível de Ensino')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('salario_vs_formacao.png')
plt.show()
plt.close()

# 4. Gráfico 3D Interativo (usando Plotly)
fig_3d = px.scatter_3d(df.dropna(subset=['Faixa_salarial_num', 'Tempo de experiência na área de dados', 'Nível de ensino alcançado']), 
                       x='Faixa_salarial_num',
                       y='Tempo de experiência na área de dados',
                       z='Nível de ensino alcançado',
                       color='Área de formação acadêmica',
                       hover_name='Cargo atual',
                       height=800,
                       title='Salário vs Experiência vs Nível Educacional')
fig_3d.update_layout(scene = dict(
                        xaxis_title='Salário (R$)',
                        yaxis_title='Experiência',
                        zaxis_title='Nível Educacional'))
fig_3d.write_html('grafico_3d_interativo.html')
fig_3d.show()

# 5. Heatmap de Correlação
plt.figure(figsize=(10,6))
sns.heatmap(numeric_df_heatmap.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlações')
plt.tight_layout()
plt.savefig('heatmap_correlacao.png')
plt.show()
plt.close()

# 6. Distribuição por Gênero e Raça
fig_dist_gen_raca, ax_gen_raca = plt.subplots(1,2, figsize=(18,6))
sns.countplot(x='Gênero do profissional', data=df, ax=ax_gen_raca[0])
ax_gen_raca[0].set_title('Distribuição por Gênero')
ax_gen_raca[0].tick_params(axis='x', rotation=45)

sns.countplot(x='Cor/Raça/Etnia', data=df, ax=ax_gen_raca[1])
ax_gen_raca[1].set_title('Distribuição por Raça/Etnia')
ax_gen_raca[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('distribuicao_genero_raca.png')
plt.show()
plt.close(fig_dist_gen_raca)

# 7. Sunburst Chart Hierárquico (Plotly)
# Removendo NaNs das colunas de path para evitar erro no sunburst
sunburst_df = df.dropna(subset=['Nível de ensino alcançado', 'Área de formação acadêmica', 'Cargo atual', 'Faixa_salarial_num'])
fig_sunburst = px.sunburst(sunburst_df, 
                           path=['Nível de ensino alcançado', 'Área de formação acadêmica', 'Cargo atual'],
                           values='Faixa_salarial_num',
                           height=800,
                           title='Hierarquia: Nível de Ensino, Área de Formação, Cargo e Salário')
fig_sunburst.write_html('sunburst_chart.html')
fig_sunburst.show()

# 8. Distribuição Geográfica
plt.figure(figsize=(12,6))
df['UF onde mora'].value_counts().plot(kind='bar')
plt.title('Distribuição Geográfica dos Profissionais')
plt.xlabel('Estado (UF)')
plt.ylabel('Contagem')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('distribuicao_geografica.png')
plt.show()
plt.close()

# 9. Senioridade vs Salário
plt.figure(figsize=(12,6))
sns.boxplot(x='Nível de senioridade', y='Faixa_salarial_num', data=df)
plt.title('Salário por Nível de Senioridade')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('senioridade_vs_salario.png')
plt.show()
plt.close()

# 10. Pairplot Multivariado
# Preparar DataFrame para pairplot, limpando inf e NaN
cols_for_pairplot = ['Faixa_salarial_num', 'Oportunidade de aprendizado', 
                     'Reputação da empresa', 'Nível de senioridade']
pairplot_df = df[cols_for_pairplot].copy()
pairplot_df.replace([np.inf, -np.inf], np.nan, inplace=True)
pairplot_df.dropna(inplace=True)

# Verificar se há dados suficientes após o dropna para o pairplot
if not pairplot_df.empty:
    sns.pairplot(pairplot_df, diag_kind='kde', corner=True) # Adicionado diag_kind e corner para melhor visualização
    plt.suptitle('Análise Multivariada das Relações entre Variáveis Selecionadas', y=1.02)
    plt.tight_layout()
    plt.savefig('pairplot_multivariado.png')
    plt.show()
    plt.close()
else:
    print("Não há dados suficientes para gerar o Pairplot após remover NaNs.")

print("Análise exploratória concluída e gráficos salvos.")

